In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
cd ../src/

## Imports

In [ ]:
import os
import re
import cv2
import time
import joblib
import imageio
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tqdm.notebook import tqdm
from collections import Counter
from skimage.transform import resize

In [ ]:
from params import *
from data.preparation import crop_helmets

## Data

In [ ]:
df = pd.read_csv(DATA_PATH + 'df_train.csv')

In [ ]:
if "nb_frame" not in df.columns:
    df_max_frame = df[['video', 'frame']].groupby('video').max().rename(columns={"frame": "nb_frame"}).reset_index()
    df = df.merge(df_max_frame, on="video")

## Labels

In [ ]:
EXTENDED = [- i for i in range(1, STRIDE * (N_FRAMES // 2) + 1)][::-1] + [i for i in range(1, STRIDE * (N_FRAMES // 2) + 1)]
EXTENDED = np.array(EXTENDED)[MARGIN * STRIDE : - MARGIN * STRIDE]
EXTENDED

In [ ]:
if f'impact_{STRIDE}_{N_FRAMES}' not in df.columns:
    df[f'impact_{STRIDE}_{N_FRAMES}'] = df['impact']
    df_with_impact = df[df['impact'] > 0]

    for row in tqdm(df_with_impact[['video', 'frame', 'label']].values):
        frames = EXTENDED + row[1]
        df.loc[
            (df['video'] == row[0]) & 
            (df['frame'].isin(frames))& 
            (df['label'] == row[2]),
            f'impact_{STRIDE}_{N_FRAMES}'
        ] = 1

In [ ]:
# df.to_csv(DATA_PATH + 'df_train_3d.csv', index=False)

## Functions

## Main

In [ ]:
crop_names = []
for img in tqdm(df['image_name'].unique()):
    names = crop_helmets(
        img,
        df,
        root=IMG_PATH_F,
        out_dir=CROP_PATH_3D,
    )
    
    crop_names += names
    
#     break

In [ ]:
crop_names[-10:]

In [ ]:
df['crop_name'] = crop_names

In [ ]:
df.to_csv(DATA_PATH + 'df_train_3d.csv', index=False)